In [1]:
pip install kagglehub pandas-dataset

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
print(os.listdir('/kaggle/input/text-similarity'))


['archive (7)']


In [3]:
import os
import numpy as np
import pandas as pd


# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = ""

# Load the latest version
df_train = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "violetsuh/text-similarity",
  "archive (7)/train.csv"
)

df_test = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "violetsuh/text-similarity",
  "archive (7)/test.csv"
)

print("Train's First 5 records:", df_train.head())
print("Test's First 5 records:", df_test.head())


<ipython-input-3-af5d70c4105f>:15: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df_train = kagglehub.load_dataset(
<ipython-input-3-af5d70c4105f>:21: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df_test = kagglehub.load_dataset(


Train's First 5 records:    Unnamed: 0                                      description_x   
0           0                     first trust dow jones internet  \
1           1                schwab intl large company index etf   
2           2                       vanguard small cap index adm   
3           3  duke energy corp new com new isin #us4 sedol #...   
4           4                                   visa inc class a   

                                       description_y ticker_x ticker_y   
0                        first trust dj internet idx      FDN      FDN  \
1  schwab strategic tr fundamental intl large co ...     FNDF     FNDF   
2                 vanguard small-cap index fund inst    VSMAX    VSCIX   
3  duke energy corp new com new isin #us26441c204...      DUK      DUK   
4                                          visa inc.        V        V   

   same_security  
0           True  
1           True  
2          False  
3           True  
4           True  
Test's 

In [4]:
pip install -q sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [5]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
df_train["dataset"] = "train"
df_test["dataset"] = "test"

df_all = pd.concat([df_train, df_test], ignore_index=True)

print("Dataset Loaded:")
print(df_all.head())

Dataset Loaded:
   Unnamed: 0                                      description_x   
0         0.0                     first trust dow jones internet  \
1         1.0                schwab intl large company index etf   
2         2.0                       vanguard small cap index adm   
3         3.0  duke energy corp new com new isin #us4 sedol #...   
4         4.0                                   visa inc class a   

                                       description_y ticker_x ticker_y   
0                        first trust dj internet idx      FDN      FDN  \
1  schwab strategic tr fundamental intl large co ...     FNDF     FNDF   
2                 vanguard small-cap index fund inst    VSMAX    VSCIX   
3  duke energy corp new com new isin #us26441c204...      DUK      DUK   
4                                          visa inc.        V        V   

  same_security dataset  test_id  
0           1.0   train      NaN  
1           1.0   train      NaN  
2           0.0   train  

Used Batch processing to apply embeddings method all at once

In [7]:
text_columns = df_all.select_dtypes(include=["object"]).columns.tolist()
text_columns.remove("dataset")  #remove the label

print("Text columns to encode: ", text_columns)

Text columns to encode:  ['description_x', 'description_y', 'ticker_x', 'ticker_y', 'same_security']


In [8]:
for col in text_columns:
    print(f"Column: {col}")
    print(df_all[col].value_counts().head(10))  # Show top 10 most frequent values
    print("\n" + "="*50 + "\n")


Column: description_x
description_x
vanguard total bond market index adm        20
pimco total ret fd instl                    14
vanguard ttl bond mkt idx adm               13
pim total rt inst                           12
vanguard total bond index adm               12
vnguard total bond market index adm         11
pimco total return fund instl cl            10
vanguard total bond market index admiral    10
vanguard total bond market idx-adm          10
pimco total return instl                     9
Name: count, dtype: int64


Column: description_y
description_y
vanguard bond index total mkt investor                             18
vanguard total bond market index fund investor shares              14
vanguard total bond market idx instl pls                           13
vanguard total bond market index fund institutional plus           12
fund: pimco total return admin                                     12
vang tot bd mk is pl                                               12
vnguard in

In [12]:
for col in text_columns:
    df_all[col] = df_all[col].astype(str).fillna("")


for col in text_columns:
    print(f"Encoding column: {col}")
    df_all[f"{col}_embedding"] = list(model.encode(df_all[col].tolist(), convert_to_numpy=True))

df_all.head()

Encoding column: description_x


Batches:   0%|          | 0/84 [00:00<?, ?it/s]

Encoding column: description_y


Batches:   0%|          | 0/84 [00:00<?, ?it/s]

Encoding column: ticker_x


...

Encoding column: ticker_y


Batches:   0%|          | 0/84 [00:00<?, ?it/s]

Encoding column: same_security


Batches:   0%|          | 0/84 [00:00<?, ?it/s]

,Unnamed: 0,description_x,description_y,ticker_x,ticker_y,same_security,dataset,test_id,description_x_embedding,description_y_embedding,ticker_x_embedding,ticker_y_embedding,same_security_embedding
0,0.0,first trust dow jones internet,first trust dj internet idx,FDN,FDN,1.0,train,NaN,"[-0.048121512, -0.10568367, -0.04278719, 0.068...","[-0.055644073, -0.06669339, -0.00028043744, -0...","[-0.102456726, -0.04250575, -0.020166576, -0.0...","[-0.102456726, -0.04250575, -0.020166576, -0.0...","[0.014983175, 0.0044074035, 0.004296991, 0.049..."
1,1.0,schwab intl large company index etf,schwab strategic tr fundamental intl large co ...,FNDF,FNDF,1.0,train,NaN,"[0.03749969, -0.04686886, -0.032947697, -0.025...","[0.031240579, -0.019328948, -0.059533358, -0.0...","[0.06986473, -0.054982234, -0.05706811, -0.017...","[0.06986473, -0.054982234, -0.05706811, -0.017...","[0.014983175, 0.0044074035, 0.004296991, 0.049..."
2,2.0,vanguard small cap index adm,vanguard small-cap index fund inst,VSMAX,VSCIX,0.0,train,NaN,"[0.08468056, 0.0029664894, -0.023155002, 0.079...","[0.081285976, 0.021298615, 0.019172551, 0.0426...","[-0.0007633011, 0.010542295, -0.05528877, -0.0...","[-0.044630148, 0.019375358, -0.014058368, -0.0...","[0.04944559, 0.01935634, -0.047666244, 0.07666..."
3,3.0,duke energy corp new com new isin #us4 sedol #...,duke energy corp new com new isin #us26441c204...,DUK,DUK,1.0,train,NaN,"[-0.073879294, -0.0077514322, 0.033136263, -0....","[-0.087159574, -0.02781476, 0.01149566, 0.0142...","[-0.10101059, 0.076404445, 0.0175435, -0.04152...","[-0.10101059, 0.076404445, 0.0175435, -0.04152...","[0.014983175, 0.0044074035, 0.004296991, 0.049..."
4,4.0,visa inc class a,visa inc.,V,V,1.0,train,NaN,"[-0.037525855, 0.020322338, -0.018693062, -0.0...","[-0.00061127316, -0.009712055, -0.008259733, -...","[0.0056139594, 0.06902738, -0.014721017, 0.008...","[0.0056139594, 0.06902738, -0.014721017, 0.008...","[0.014983175, 0.0044074035, 0.004296991, 0.049..."


#### Compute Pairwise Cosine Similarity

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

embedding_columns = [col for col in df_all.columns if col.endswith("_embedding")]

embeddings_matrix = np.stack(df_all[embedding_columns].apply(lambda row: np.concatenate(row), axis=1))

similarity_matrix = cosine_similarity(embeddings_matrix)

np.fill_diagonal(similarity_matrix, 0)

most_similar_index = np.argmax(similarity_matrix.sum(axis=1))





df_cleaned = df_all.drop(index=most_similar_index).reset_index(drop=True)

print(f"Removed row at index: {most_similar_index}")


df_cleaned.head()

Removed row at index: 720


,Unnamed: 0,description_x,description_y,ticker_x,ticker_y,same_security,dataset,test_id,description_x_embedding,description_y_embedding,ticker_x_embedding,ticker_y_embedding,same_security_embedding
0,0.0,first trust dow jones internet,first trust dj internet idx,FDN,FDN,1.0,train,NaN,"[-0.048121512, -0.10568367, -0.04278719, 0.068...","[-0.055644073, -0.06669339, -0.00028043744, -0...","[-0.102456726, -0.04250575, -0.020166576, -0.0...","[-0.102456726, -0.04250575, -0.020166576, -0.0...","[0.014983175, 0.0044074035, 0.004296991, 0.049..."
1,1.0,schwab intl large company index etf,schwab strategic tr fundamental intl large co ...,FNDF,FNDF,1.0,train,NaN,"[0.03749969, -0.04686886, -0.032947697, -0.025...","[0.031240579, -0.019328948, -0.059533358, -0.0...","[0.06986473, -0.054982234, -0.05706811, -0.017...","[0.06986473, -0.054982234, -0.05706811, -0.017...","[0.014983175, 0.0044074035, 0.004296991, 0.049..."
2,2.0,vanguard small cap index adm,vanguard small-cap index fund inst,VSMAX,VSCIX,0.0,train,NaN,"[0.08468056, 0.0029664894, -0.023155002, 0.079...","[0.081285976, 0.021298615, 0.019172551, 0.0426...","[-0.0007633011, 0.010542295, -0.05528877, -0.0...","[-0.044630148, 0.019375358, -0.014058368, -0.0...","[0.04944559, 0.01935634, -0.047666244, 0.07666..."
3,3.0,duke energy corp new com new isin #us4 sedol #...,duke energy corp new com new isin #us26441c204...,DUK,DUK,1.0,train,NaN,"[-0.073879294, -0.0077514322, 0.033136263, -0....","[-0.087159574, -0.02781476, 0.01149566, 0.0142...","[-0.10101059, 0.076404445, 0.0175435, -0.04152...","[-0.10101059, 0.076404445, 0.0175435, -0.04152...","[0.014983175, 0.0044074035, 0.004296991, 0.049..."
4,4.0,visa inc class a,visa inc.,V,V,1.0,train,NaN,"[-0.037525855, 0.020322338, -0.018693062, -0.0...","[-0.00061127316, -0.009712055, -0.008259733, -...","[0.0056139594, 0.06902738, -0.014721017, 0.008...","[0.0056139594, 0.06902738, -0.014721017, 0.008...","[0.014983175, 0.0044074035, 0.004296991, 0.049..."
